# Modelling Laban Features to 1024 element Spectrogram Represetnation

In [1]:
#Importing the required packages
from __future__ import absolute_import, division, print_function, unicode_literals
import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'


import collections
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import librosa
import librosa.display
import IPython.display as ipd
import scipy
import tensorflow as tf
import pandas as pd
import mdn


from pickle import dump, load
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Activation, LSTM, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks.callbacks import ModelCheckpoint, CSVLogger
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, minmax_scale, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras import regularizers


/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV

In [2]:
#Lists the local devices (GPUs) 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11045442384184296929
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16678989469193219519
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12004632167
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 2652234470268638539
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:82:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15226798049716725523
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
#Prints the numbers of GPUs availible/selected
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


## Prepare Data

In [15]:
#Loading in the Spectrogram Representations and Motion Features
PATH = os.getcwd()

SpectrogramPath = PATH+'/SMC_dataset1/Spectrograms1024.csv'
MotionPath = PATH+'/SMC_dataset1/motion_features8x5.npy'

#Load Motion features
MotionFeat = np.load(MotionPath)

#Load Motion features
Spectro = pd.read_csv(SpectrogramPath, sep=',' ,header=None)



In [16]:
#Feature Scaling/Normalisation of Data 

#Select 20% of data for validation set
train_laban, validate_laban, train_spec, validate_spec = train_test_split(MotionFeat, Spectro, test_size=0.2, random_state=1, shuffle=False)


#Scale to between 0 and 1
scaleObject = MinMaxScaler(feature_range=(0,1))
train_spec = scaleObject.fit_transform(train_spec)
validate_spec = scaleObject.transform(validate_spec)

#The scaleObject is needed to transform the spectrogram representations back to the appropriote range
dump(scaleObject, open('scaleObject.pkl', 'wb'))

## Neural Network

In [17]:
#Parameters
batch_size=8

# number of mixture components
N_MIXES = 10  

# number of real-values predicted by each mixture component
OUTPUT_DIMS = 1024  

In [10]:
model = Sequential()

#input to model is 5 vectors of 8 features
model.add(LSTM(64, activation = 'softsign', return_sequences=True, batch_input_shape=(batch_size, 8, 5), dropout=0.2))
model.add(LSTM(128, activation = 'softsign', return_sequences=False, dropout=0.2))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1024, activation = 'relu',  activity_regularizer=regularizers.l1(0.01)))
model.add(mdn.MDN(OUTPUT_DIMS, N_MIXES))
model.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMS,N_MIXES), optimizer=Adam(learning_rate=0.0001))

model.summary()

Instructions for updating:
The `event_size` property is deprecated.  Use `num_categories` instead.  They have the same value, but `event_size` is misnamed.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (8, 8, 64)                17920     
_________________________________________________________________
lstm_5 (LSTM)                (8, 128)                  98816     
_________________________________________________________________
dense_4 (Dense)              (8, 256)                  33024     
_________________________________________________________________
dense_5 (Dense)              (8, 512)                  131584    
_________________________________________________________________
dense_6 (Dense)              (8, 1024)                 525312    
__

In [11]:
#Best new validation models are saved as 'Model.hdf5' and loss values saved to 'log.csv'
csv_logger = CSVLogger('log.csv', append=True, separator=',')
filepath="MDN/Model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, csv_logger]

history = model.fit(train_laban, train_spec, batch_size=batch_size, epochs=1000, validation_data=(validate_laban, validate_spec), callbacks=callbacks_list, verbose=0, shuffle=False)




Epoch 00001: val_loss improved from inf to -691.13340, saving model to MDN/Model.hdf5

Epoch 00002: val_loss improved from -691.13340 to -783.33918, saving model to MDN/Model.hdf5

Epoch 00003: val_loss improved from -783.33918 to -825.91839, saving model to MDN/Model.hdf5

Epoch 00004: val_loss improved from -825.91839 to -833.54581, saving model to MDN/Model.hdf5

Epoch 00005: val_loss improved from -833.54581 to -863.06394, saving model to MDN/Model.hdf5

Epoch 00006: val_loss improved from -863.06394 to -863.53563, saving model to MDN/Model.hdf5

Epoch 00007: val_loss improved from -863.53563 to -887.00075, saving model to MDN/Model.hdf5

Epoch 00008: val_loss improved from -887.00075 to -905.56690, saving model to MDN/Model.hdf5

Epoch 00009: val_loss did not improve from -905.56690

Epoch 00010: val_loss improved from -905.56690 to -913.55067, saving model to MDN/Model.hdf5

Epoch 00011: val_loss improved from -913.55067 to -915.72376, saving model to MDN/Model.hdf5

Epoch 00012


Epoch 00129: val_loss did not improve from -1144.13997

Epoch 00130: val_loss improved from -1144.13997 to -1149.88056, saving model to MDN/Model.hdf5

Epoch 00131: val_loss did not improve from -1149.88056

Epoch 00132: val_loss improved from -1149.88056 to -1164.75296, saving model to MDN/Model.hdf5

Epoch 00133: val_loss did not improve from -1164.75296

Epoch 00134: val_loss did not improve from -1164.75296

Epoch 00135: val_loss did not improve from -1164.75296

Epoch 00136: val_loss did not improve from -1164.75296

Epoch 00137: val_loss did not improve from -1164.75296

Epoch 00138: val_loss did not improve from -1164.75296

Epoch 00139: val_loss did not improve from -1164.75296

Epoch 00140: val_loss did not improve from -1164.75296

Epoch 00141: val_loss improved from -1164.75296 to -1174.35272, saving model to MDN/Model.hdf5

Epoch 00142: val_loss did not improve from -1174.35272

Epoch 00143: val_loss did not improve from -1174.35272

Epoch 00144: val_loss did not improve f


Epoch 00268: val_loss did not improve from -1229.77830

Epoch 00269: val_loss did not improve from -1229.77830

Epoch 00270: val_loss did not improve from -1229.77830

Epoch 00271: val_loss did not improve from -1229.77830

Epoch 00272: val_loss did not improve from -1229.77830

Epoch 00273: val_loss did not improve from -1229.77830

Epoch 00274: val_loss did not improve from -1229.77830

Epoch 00275: val_loss did not improve from -1229.77830

Epoch 00276: val_loss did not improve from -1229.77830

Epoch 00277: val_loss did not improve from -1229.77830

Epoch 00278: val_loss did not improve from -1229.77830

Epoch 00279: val_loss did not improve from -1229.77830

Epoch 00280: val_loss did not improve from -1229.77830

Epoch 00281: val_loss did not improve from -1229.77830

Epoch 00282: val_loss did not improve from -1229.77830

Epoch 00283: val_loss did not improve from -1229.77830

Epoch 00284: val_loss did not improve from -1229.77830

Epoch 00285: val_loss did not improve from -122


Epoch 00415: val_loss did not improve from -1229.77830

Epoch 00416: val_loss did not improve from -1229.77830

Epoch 00417: val_loss did not improve from -1229.77830

Epoch 00418: val_loss did not improve from -1229.77830

Epoch 00419: val_loss did not improve from -1229.77830

Epoch 00420: val_loss did not improve from -1229.77830

Epoch 00421: val_loss did not improve from -1229.77830

Epoch 00422: val_loss did not improve from -1229.77830

Epoch 00423: val_loss did not improve from -1229.77830

Epoch 00424: val_loss did not improve from -1229.77830

Epoch 00425: val_loss did not improve from -1229.77830

Epoch 00426: val_loss did not improve from -1229.77830

Epoch 00427: val_loss did not improve from -1229.77830

Epoch 00428: val_loss did not improve from -1229.77830

Epoch 00429: val_loss did not improve from -1229.77830

Epoch 00430: val_loss did not improve from -1229.77830

Epoch 00431: val_loss did not improve from -1229.77830

Epoch 00432: val_loss did not improve from -122


Epoch 00562: val_loss did not improve from -1229.77830

Epoch 00563: val_loss did not improve from -1229.77830

Epoch 00564: val_loss did not improve from -1229.77830

Epoch 00565: val_loss did not improve from -1229.77830

Epoch 00566: val_loss did not improve from -1229.77830

Epoch 00567: val_loss did not improve from -1229.77830

Epoch 00568: val_loss did not improve from -1229.77830

Epoch 00569: val_loss did not improve from -1229.77830

Epoch 00570: val_loss did not improve from -1229.77830

Epoch 00571: val_loss did not improve from -1229.77830

Epoch 00572: val_loss did not improve from -1229.77830

Epoch 00573: val_loss did not improve from -1229.77830

Epoch 00574: val_loss did not improve from -1229.77830

Epoch 00575: val_loss did not improve from -1229.77830

Epoch 00576: val_loss did not improve from -1229.77830

Epoch 00577: val_loss did not improve from -1229.77830

Epoch 00578: val_loss did not improve from -1229.77830

Epoch 00579: val_loss did not improve from -122


Epoch 00709: val_loss did not improve from -1229.77830

Epoch 00710: val_loss did not improve from -1229.77830

Epoch 00711: val_loss did not improve from -1229.77830

Epoch 00712: val_loss did not improve from -1229.77830

Epoch 00713: val_loss did not improve from -1229.77830

Epoch 00714: val_loss did not improve from -1229.77830

Epoch 00715: val_loss did not improve from -1229.77830

Epoch 00716: val_loss did not improve from -1229.77830

Epoch 00717: val_loss did not improve from -1229.77830

Epoch 00718: val_loss did not improve from -1229.77830

Epoch 00719: val_loss did not improve from -1229.77830

Epoch 00720: val_loss did not improve from -1229.77830

Epoch 00721: val_loss did not improve from -1229.77830

Epoch 00722: val_loss did not improve from -1229.77830

Epoch 00723: val_loss did not improve from -1229.77830

Epoch 00724: val_loss did not improve from -1229.77830

Epoch 00725: val_loss did not improve from -1229.77830

Epoch 00726: val_loss did not improve from -122


Epoch 00856: val_loss did not improve from -1229.77830

Epoch 00857: val_loss did not improve from -1229.77830

Epoch 00858: val_loss did not improve from -1229.77830

Epoch 00859: val_loss did not improve from -1229.77830

Epoch 00860: val_loss did not improve from -1229.77830

Epoch 00861: val_loss did not improve from -1229.77830

Epoch 00862: val_loss did not improve from -1229.77830

Epoch 00863: val_loss did not improve from -1229.77830

Epoch 00864: val_loss did not improve from -1229.77830

Epoch 00865: val_loss did not improve from -1229.77830

Epoch 00866: val_loss did not improve from -1229.77830

Epoch 00867: val_loss did not improve from -1229.77830

Epoch 00868: val_loss did not improve from -1229.77830

Epoch 00869: val_loss did not improve from -1229.77830

Epoch 00870: val_loss did not improve from -1229.77830

Epoch 00871: val_loss did not improve from -1229.77830

Epoch 00872: val_loss did not improve from -1229.77830

Epoch 00873: val_loss did not improve from -122


Epoch 00998: val_loss did not improve from -1277.78309

Epoch 00999: val_loss did not improve from -1277.78309

Epoch 01000: val_loss did not improve from -1277.78309
